# Extracción de Tweets

En esta notebook se van a revisar los pasos para lograr un buen minado de tweets, esto incluye: que es la API de Twitter, las herramientas u operadores que facilita para armar quieres, obtener un dataset inicial de tweets y como refinar iterativamente los queries.

## Requerimientos

### Credenciales para la API de Twitter
Para obtener tweets con la [API Oficial de Twitter](https://developer.twitter.com/en/docs/twitter-api) es necesario registrarse en el [portal para desarrolladores de Twitter](https://developer.twitter.com/en/docs/developer-portal/overview) para así dar de alta un proyecto y que se te asignen las [credenciales necesarias](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api).

Hay que aclarar que **según el tipo de credenciales y le metodo de autenticación serán diferente la [cantidad de tweets que podremos obtener en cierto tiempo](https://developer.twitter.com/en/docs/twitter-api/rate-limits#v2-limits), que [operadores/filtros podemos usar](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list) y cuantos días hacia atras podremos minar**. Tambien cambia la cantidad máxima de tweets que se pueden obtener en un mes y el largo máximo de los queries. 
Por ejemplo, al mes:
- *Essential*: 500k tweets en un mes y 512 caracteres por query.
- *Elevated*: 2M tweets en un mes y 512 caracteres por query.
- *Academic*: 10M tweets en un mes y 1024 caracteres por query.

Cualquier otra diferencia entre nivel de credenciales, se puede [ver aquí](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level)

Notas:
- Dado que nuestra app solo necesita permisos de lectura para contenido publico, nos interesa obtener la credencial de tipo **Bearer Token**, ya que es el metodo de autenticación que más tweets nos permite obtener en menos tiempo.
- Se usara la versión 2 de la [API de Twitter](https://developer.twitter.com/en/docs/twitter-api).
- El ejericio se puede realizar con las credenciales de nivel [*Elevated*](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level), pero en caso de estar trabajando en un proyecto con fines academicos o de estudio, se podria [aplicar de manera gratuita a credenciales Academicas](https://developer.twitter.com/en/products/twitter-api/academic-research) y así trabajar con limites más amplios y operadores avanzados.

### Instalar dependencias de Python
Las dependencias principales son `twarc`, `pandas` y `numpy`. Las ultimas dos son para manipular y trabajar con los datos, mientrar que [`twarc` es el paquete](https://developer.twitter.com/en/docs/twitter-api/rate-limits) que nos facilitara el minado, ya que se encarga automaticamente de obtener todos los atributos de los tweets y tambien de manejar los tiempos de espera cuando se llega a [rate limits de la API](https://developer.twitter.com/en/docs/twitter-api/rate-limits).

In [ ]:
# Indispensables
!pip install twarc==2.9.2
!pip install pandas==1.4.1
!pip install numpy==1.22.3

# Para mejor interacción gráfica
tqdm==4.62.2
pendulum==2.1.2

## Imports & Credenciales

In [29]:
import os
import pandas as pd
from tqdm import tqdm

from twarc.client2 import Twarc2
from twarc.expansions import TWEET_FIELDS
from twarc.expansions import ensure_flattened

Ingresa tu propio Bearer Token o comentalo, y descomenta el resto de atributos para con tus propias API Keys y Access Tokens.

In [30]:
CREDENTIALS_TWITTER = {
    'bearer_token': "Enter your own Bearer token",

    # 'api_key': "Enter your own API Key",
    # 'api_secret_key': "Enter your own API Secret Key",
    # 'access_token': "Enter your own access_token",
    # 'access_token_secret': "Enter your own access_token_secret"
}

CREDENTIALS_TWITTER = {
    'bearer_token': "AAAAAAAAAAAAA0AAAAAAAAFBIPwEAAAAAxt1ACpkLKEy%2FwL4KvhyxKCLAIbA%3D1ORuBxC0mD6fY47zNqdNAXICGQboR5EQu0hxie6P3EAeEFo6ZW"
}

## Minado de Tweets

La API de Twitter permite obtener diferentes tipos de información de los tweets y usuarios, según el tipo de operadores y queries utilizados. Para fines de este notebook, nos centraremos en obtener tweets públicos dados un intervalo de tiempo y palabras claves (queries).


### Endpoints & Limites
Para [buscar tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/introduction) hay dos endpoints, [Recent Search](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/recent-search) y  [Full Archive](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/full-archive-search).
- **Recent Search**: Nos permite realizar 450 requests (pedidos) a la API en una ventana de 15 minutos. Con un maximo de 100 tweets por request. Pero solo podriamos obtener tweets que hayan sido publicados en los utlimos 7 dias.
- **Full Archive**: Podremos obtener tweets publicados desde el inicio de la red social, pero solo serán 300 requests en la ventana de 15 minutos, con hasta 500 tweets por cada request. Hay que aplicar a las [credenciales academicas](https://developer.twitter.com/en/products/twitter-api/academic-research) para poder utilizar este endpoint.

**Nota**: Nosotros no tenemos que preocuparnos de que el codigo de error una vez se alcance estos limites, ya que el paquete twarc se encarga de pausar la obtención de tweets una vez que se alcanzó el limite de 450 (o 300) request en la ventana de 15 minutos, para luego continuar una vez haya pasado el tiempo necesario. Y si se llega al limite de tweets en un mes, la función va a terminar.



### Operadores

En esta sección vamos a ver de manera general que son los operadores, pero la lista y descripción completa de los operadores se puede [encontrar aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query).

Hay dos tipos de operadores: 
- **Standalone**: Se pueden usar solos o en conjunción de otros. Por ejemplo, buscar tweets con un hashtag en especifico: `#migrantes`.
- **Conjunction-required**: Es necesario que esten junto a minimo un operador *standalone*. Por ejemplo, buscar tweets con un hastag en especifico pero que incliuyan imagenes y sean retweets: `#migrantes has:media is:retweet`

Ademas, como se menciono en la sección de "Credenciales para la API de Twitter", existen operadores *core*, que son accesibles con cualquier nivel de acceso y tambien estan los *advanced*, que solo se pueden utilizar con un acceso academico. 

#### Operadores Logicos

- **AND**: Obtiene tweets que cumplan con los dos operadores, se logra dejando un espacio en blanco entre ellos. 
    - Ejemplo, obtener tweets que contienen la palabra *politicos* y el hashtag *#corruptos*: `politicos #corruptos`.
- **OR**: Obtiene tweets que cumplan con alguno de los dos operadores. Hay que añadir el string " OR " entre los operadores. 
    - Ejemplo, tweets que contengan la palabra *migrantes* o *inmigrantes*: `migrantes OR inmigrantes`.
- **NOT**: Obtiene tweets que no contengan el operador o la keyword negada. Se logra añadiendo un guión medio "-" antes del operador. 
    - Ejemplo, obtener tweets con la palabra *politicos* pero sin la palabra *corruptos*: `politicos -corrupts`
    - Ejemplo, obtener tweets con la palabra *migrantes pero que no sean retweets*: `migrantes -is:retweets`
- **Grouping**: Sirve para agrupar operadores logicos, y hay que encerrar los operadores entre parentesis. Un grupo no puede ser negado.
    - Ejemplo, obtener tweets con la palabra *migrantes* y alguna de las palabras *llegan* o *salen*: `migrantes AND (llegan OR salen)`

**Nota**: A menos de que haya parentesis para especificar el orden de operadores, primero se resuelven aquellos que son *AND* y luego los *OR*. [Más aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#boolean).

#### Más Operadores

- **keyword**: Hace match a tweets que contengan un string en especifico (no se pueden usar wildcards). No es sensible a si hay caracteres en mayusculas o minusculas, tampoco a acentos y otros caracteres como ñ. Ejemplo: `migrantes OR inmigrantes`.
- **"exact phrase"**: Parecedido al anterior, pero permite tener espacios y multiples tokens. Tienen que estar envueltro por comillas dobles, ejemplo: `"ola migrante"`.
- **#**: Hace match a tweets que tengan el hashtag incluido. Ejemplo: `#migraresunderecho`.
- **@**: Hace match a tweets que mencionen a los usuarios incluidos. Ejemplo: `@el_BID`.
- **place_country**: Obtiene tweets que sean geolocalizables a cierto pais. Hay que pasarle el código [ISO del país](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2). Ejemplo: `place_country:MX`.
- **lang**: Obtiene tweets que esten escrito en el lenguaje indicado y tiene que estar junto con un standalone operator. Ejemplo: `migrantes lang:es`.
- **is:retweet**: Obtiene solo tweets que sean retweets y tiene que estar junto con un standalone operator. Ejemplo: `migrantes is:retweet`.

La lista completa de operadores esta [aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list).

#### Ejercicio de Minado

La siguiente función recibe una lista de queries y un intervalo de tiempo para guardar los tweets en un archivo JSONL.

**Nota**: Un archivo de extension .jsonl guarda un objeto json por cada linea separados por solo el salto de linea, no comas u otro separador.

In [ ]:
def get_tweets(twarc_config,
               queries_list,
               output_file,
               since_date, until_date,
               is_academic=False):
    """Function in charge of scrape tweets from the 
    official Twitter API, using the library named Twarc.

    Args:
        twarc_config (dict): Dictionary with the Twitter API credentials.
        queries_list (list[str]): List of queries to scrape.
        output_file (str): Path to the file in which to store the results.
        since_date (datetime): Start of the time span to scrape.
        until_date (datetime): End of the time span to scrape.
        is_academic (bool, optional): If the credentials has Research 
                                      Academic access level.
    """

    # Instiate the Twarc Client
    twarc_client = Twarc2(**twarc_config)

    # Make some tweaks for using the research credentials
    max_size = 100
    tweet_fields = TWEET_FIELDS.copy()
    search_func = twarc_client.search_recent
    if(is_academic):
        search_func = twarc_client.search_all
        max_size = 500

        # Remove the context_annotations attr to
        # scrape 500 tweets per request
        tweet_fields.remove('context_annotations')

    tweet_fields = ','.join(tweet_fields)

    # Keep in memory the wanted attrs
    return_values = []

    with open(output_file, 'a') as pages_file:
        for query in tqdm(queries_list):

            search_results = search_func(query=query,
                                         start_time=since_date,
                                         end_time=until_date,
                                         tweet_fields=tweet_fields,
                                         max_results=max_size)

            # Write all the obtained tweets
            for page in search_results:

                # Write one by one the tweets
                for tweet in ensure_flattened(page):
                    json.dump(tweet, pages_file)
                    pages_file.write('\n')

Se declara la lista de queries, el intervalo de fecha (para fines de la prueba, es 6hrs hacia atras de la hora actual) y si las credenciales que se estan utilizando tienen un nivel de acceso academico. Esto ultimo es util para pedir que cada request tenga 500 tweets.

In [ ]:
lst_queries = ['migrantes', 'inmigrantes', 'emigrantes']

date_start = pendulum

